# Demo

## Artifically generated encode data

In [1]:
def generate_data(data, bit_rate_step=50, max_bitrate = 2500, min_birate = 50, widths=[1280,  960, 1920,  640,  416,  768], heights = [720,  540, 1080,  360,  234,  432], col_names=['e_height','e_width', 't_average_bitrate']):
    generated=pd.DataFrame()
    for bitrate in range(min_birate,max_bitrate,bit_rate_step):
        for width in widths:
            for height in heights:
                data[col_names[0]]=height
                data[col_names[1]]=width
                data[col_names[2]]=bitrate
                generated = generated.append(data, ignore_index = True)
    return generated


## Highest VMAF value & Getting Encode ladder

In [2]:
def get_data_at_highest_vmaf_per_bps(data):
    best_per_bitrate = pd.DataFrame(columns = data.columns)
    for bitrate in data['t_average_bitrate'].unique():
        df_tmp = data[data['t_average_bitrate']==bitrate]
        best_vmaf = df_tmp[df_tmp['t_average_vmaf']==df_tmp['t_average_vmaf'].max()]
        best_per_bitrate = best_per_bitrate.append(best_vmaf)
    return best_per_bitrate

def get_encoding_latter(best_per_bitrate):
    sorted_df = best_per_bitrate.sort_values('t_average_vmaf')
    jnd_df = pd.DataFrame(sorted_df.head(n=1))
    for i in range(1,len(sorted_df)):
        if not (sorted_df.iloc[i,-1]-jnd_df.iloc[-1,-1])<6:
            jnd_df = jnd_df.append(sorted_df.iloc[i,:], ignore_index = True)
    return jnd_df


# Trained Model - Random Forest Regressor

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

X_train = pd.read_csv('data/training_data/X_train.csv')
X_train.drop('id', axis = 1, inplace=True)
y_train = pd.read_csv('data/training_data/y_train.csv')
y_train.drop('id', axis = 1, inplace=True)
X_test = pd.read_csv('data/test_data/X_test.csv')
X_test.drop('id', axis = 1, inplace=True)
y_test = pd.read_csv('data/test_data/y_test.csv')
y_test.drop('id', axis = 1, inplace=True)

#fill na training data
X_train = X_train.apply(lambda x: x.fillna(x.mean()),axis=0)
X_test = X_test.apply(lambda x: x.fillna(x.mean()),axis=0)

#drop variables that worsen prediction
X_train.drop(['s_storage_size', 's_duration'], axis = 1, inplace=True)
X_test.drop(['s_storage_size', 's_duration'], axis = 1, inplace=True)

regr = RandomForestRegressor(max_depth=25, random_state=0)
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)
#print(y_pred)

#have a look at the results:
print('Mean Absolute Error: %.4f' % metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error: %.4f' % metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: %.4f' % np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


# artifical "new" movie instance 
new_instance = X_test.head(n=1)

#generate the bitrates and resolutions in a step of bps
new_data = generate_data(new_instance)

#predict 
y_pred_new_data = regr.predict(new_data)
new_data['t_average_vmaf'] = y_pred_new_data

#get highest vmaf at each resolution 
best_encodings_per_bitrate = get_data_at_highest_vmaf_per_bps(new_data)

#get those with a minimum of 6 vmaf score difference (=1 jnds)
encoding_df = get_encoding_latter(best_encodings_per_bitrate)

#check when (at which bps) to switch resolution and to which resolution
print(encoding_df[['e_width', 'e_height', 't_average_bitrate']])

Mean Absolute Error: 2.2414
Mean Squared Error: 11.7897
Root Mean Squared Error: 3.4336
  e_width e_height t_average_bitrate
0     640      360                50
1     768      432               100
2     768      432               150
3     960      540               250
4     960      540               400
5    1280      720               550
6    1920     1080               800
7    1920     1080              1950
